In [53]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
def loadData(file):
    data = pd.read_csv(file,infer_datetime_format=True, parse_dates=['Date'])
    print('Raw shape: ',data.shape)
    data['date'] = data.Date #pd.to_datetime(data.date)
    data = data[ (data.Date < pd.to_datetime('2019')) & (data.Date > pd.to_datetime('2017-07')) ]
    print('Days: ',len(set(data.date)))
    return data

In [55]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [56]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = np.load('communityAggregatedMatrix.npy')
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [57]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = 'NewYorkEdgeDatewiseAggregated.csv'
#events_data =dataDir+'TaipeiEvents.csv'

In [58]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix,dates = getMatrixData()
matrix = matrix.astype(float)
matrix.shape

Raw shape:  (21380943, 4)
Days:  548
Matrix Shape:  (548, 576)


(548, 576)

In [59]:
# for i in range(matrix.shape[1]):
#     matrix[:, i] = (matrix[:, i] - matrix[:, i].min()) / (matrix[:, i].max() - matrix[:, i].min())

In [60]:
for i in range(matrix.shape[0]):
    matrix[i, :] = (matrix[i, :] - matrix[i, :].min()) / (matrix[i, :].max() - matrix[i, :].min())

In [61]:
# for i in range(matrix.shape[1]):
#     for j in range(matrix.shape[2]):
#         matrix[:, i,j] = (matrix[:, i,j] - np.mean(matrix[:, i,j])) / (np.std(matrix[:, i,j]))

In [62]:
#stations = list(set(dataRaw.start_id))
#n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [63]:
#dates = list(dataOut.index)
DOW = list(pd.to_datetime(dates).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [64]:
max(dates)

numpy.datetime64('2018-12-31T00:00:00.000000000')

In [65]:
# dataList = []

# for i in range(len(DOW)):
#     x = torch.tensor(matrix[i], dtype=torch.float)
#     y = torch.tensor(np.array([DOW[i]]), dtype=torch.long)
#     data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y)
#     dataList.append(data)

# dataList[0]

In [66]:
class autoencoder(nn.Module):
    def __init__(self,inputD,encoding_dim):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential()
        
        self.encoder.add_module("enc_0", nn.Linear(inputD,encoding_dim[0]))
        self.encoder.add_module("relu_0", nn.ReLU())
          
        for l in range(1,len(encoding_dim)):
            self.encoder.add_module("enc_"+str(l), nn.Linear(encoding_dim[l-1],encoding_dim[l]))
            self.encoder.add_module("encrelu_"+str(l), nn.ReLU())
                                    
        self.decoder = nn.Sequential()
        
        for l in range(len(encoding_dim)-1,0,-1):
            self.decoder.add_module("dec_"+str(l), nn.Linear(encoding_dim[l],encoding_dim[l-1]))
            self.decoder.add_module("decrelu_"+str(l), nn.ReLU())
            
        self.decoder.add_module("dec_0", nn.Linear(encoding_dim[0],inputD))
        self.decoder.add_module("decrelu_0", nn.Sigmoid())
        
        self.encoder.apply(self.init_weights)
        self.decoder.apply(self.init_weights)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.encoder(x)
        return x

In [67]:
matrixtensor = torch.tensor(matrix).float()

In [68]:
input_dim = matrix.shape[1]

In [69]:
encoding_dim = [500,100,20]

learning_rate=0.001
batch_size = 100
num_epochs = 100

In [70]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, matrixtensor) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [71]:
model = autoencoder(input_dim,encoding_dim).cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [72]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, _ = data
        X = X.cuda()
        # ===================forward=====================
        output = model(X)
        loss = criterion(output, X)
        MSE_loss = nn.MSELoss()(output, X)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item(), MSE_loss.item()))

epoch [1/100], loss:0.6428, MSE_loss:0.2087
epoch [2/100], loss:0.3267, MSE_loss:0.0649
epoch [3/100], loss:0.0871, MSE_loss:0.0019
epoch [4/100], loss:0.0969, MSE_loss:0.0013
epoch [5/100], loss:0.0883, MSE_loss:0.0021
epoch [6/100], loss:0.0724, MSE_loss:0.0016
epoch [7/100], loss:0.0672, MSE_loss:0.0010
epoch [8/100], loss:0.0612, MSE_loss:0.0004
epoch [9/100], loss:0.0621, MSE_loss:0.0003
epoch [10/100], loss:0.0628, MSE_loss:0.0003
epoch [11/100], loss:0.0631, MSE_loss:0.0003
epoch [12/100], loss:0.0630, MSE_loss:0.0003
epoch [13/100], loss:0.0611, MSE_loss:0.0003
epoch [14/100], loss:0.0621, MSE_loss:0.0003
epoch [15/100], loss:0.0612, MSE_loss:0.0002
epoch [16/100], loss:0.0607, MSE_loss:0.0002
epoch [17/100], loss:0.0615, MSE_loss:0.0002
epoch [18/100], loss:0.0617, MSE_loss:0.0002
epoch [19/100], loss:0.0600, MSE_loss:0.0002
epoch [20/100], loss:0.0606, MSE_loss:0.0002
epoch [21/100], loss:0.0603, MSE_loss:0.0002
epoch [22/100], loss:0.0597, MSE_loss:0.0002
epoch [23/100], los

In [73]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(548, 20)

In [74]:
# import events data
events_data =dataDir+'NewYorkEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [75]:
df_events.head()

,Type,Name,Date
0,National Holiday,New Year's Day,2016-01-01
1,National Holiday,"Martin Luther King, Jr. Day",2016-01-18
2,National Holiday,Presidents' Day,2016-02-15
3,National Holiday,Memorial Day,2016-05-30
4,National Holiday,Independence Day,2016-07-04


In [76]:
holidayDates = df_events[df_events.Type == 'National Holiday'].Date

In [77]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = pd.to_datetime(dates)
dates = [str(d.date()) for d in dates]

In [78]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [79]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [80]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [81]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [82]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.6


In [83]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.095238,0.007519,0.200000,0.0625,0.01
1,Global,0.125000,0.026316,0.125000,0.1250,0.03
2,Global,0.054054,0.037594,0.047619,0.0625,0.04
3,Global,0.093023,0.046992,0.074074,0.1250,0.05
4,Global,0.083333,0.056391,0.062500,0.1250,0.06
5,Global,0.111111,0.065789,0.078947,0.1875,0.07
6,Global,0.169492,0.071429,0.116279,0.3125,0.08
7,Global,0.200000,0.088346,0.129630,0.4375,0.10
8,Global,0.144000,0.187970,0.082569,0.5625,0.20
9,Global,0.122222,0.287594,0.067073,0.6875,0.30


In [84]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [85]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(548, 20)


In [86]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.5
Iteration 1
Itera

In [87]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.285714,0.003759,0.600000,0.1875,0.01
1,Global,0.562500,0.013158,0.562500,0.5625,0.03
2,Global,0.540541,0.020677,0.476190,0.6250,0.04
3,Global,0.465116,0.031955,0.370370,0.6250,0.05
4,Global,0.458333,0.039474,0.343750,0.6875,0.06
5,Global,0.407407,0.050752,0.289474,0.6875,0.07
6,Global,0.372881,0.060150,0.255814,0.6875,0.08
7,Global,0.314286,0.080827,0.203704,0.6875,0.10
8,Global,0.192000,0.182331,0.110092,0.7500,0.20
9,Global,0.133333,0.285714,0.073171,0.7500,0.30
